<a href="https://colab.research.google.com/github/AndreaMorbello/Progetto-IA2/blob/main/Notebook-IA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df=pd.read_csv('games.csv')